<a href="https://colab.research.google.com/github/mvrcx/SWP_Blitzer/blob/main/Landmark_Grayscales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os 
import dlib
import cv2
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
accepted_image_formats = ["jpg", "png", "jpeg"]

def get_image_urls(path):
	filenames = []
	
	# Move to the raw_data directory, to access the images
	os.chdir(path)
	
	# loop over every file in the current directory (raw_data)
	for root, directories, files in os.walk("."):
		for filename in files: 
			
			# check if file has accepted format
			if any(image_format in filename for image_format in accepted_image_formats):
				filenames.append(path+"/"+filename)

	# change back to original directory
	return filenames

In [ ]:
covered_images = get_image_urls("/content/drive/MyDrive/SWP_Blitzer/with_mask")
uncovered_images = get_image_urls("/content/drive/MyDrive/SWP_Blitzer/without_mask")
print(len(uncovered_images))

1930


In [ ]:
def get_color_of_pixel(image_url, coordinate):
    im = Image.open(image_url) 
    pix = im.load()
    return pix[coordinate[0], coordinate[1]]  
    

def get_gray_scale_of_landmarks_for(image_url):
    # Load the detector
    detector = dlib.get_frontal_face_detector()
    
    # Load the predictor
    predictor = dlib.shape_predictor("/content/drive/MyDrive/SWP_Blitzer/shape_predictor_68_face_landmarks.dat")
    
    # read the image
    img = cv2.imread(image_url)
    
    # Convert image into grayscale
    gray = cv2.cvtColor(src=img, code=cv2.COLOR_BGR2GRAY)
    
    # Use detector to find landmarks
    faces = detector(gray)
    
    gray_scales_of_landmarks = []
    
    for face in faces:
        x1 = face.left() # left point
        y1 = face.top() # top point
        x2 = face.right() # right point
        y2 = face.bottom() # bottom point
    
        # Create landmark object
        landmarks = predictor(image=gray, box=face)

        landmark_grayscales_for_face = []
        # Loop through all the points
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            if y < len(gray) and x < len(gray[0]):
                landmark_grayscales_for_face.append(gray[y, x])
        if len(landmark_grayscales_for_face) == 68:
            gray_scales_of_landmarks.append(landmark_grayscales_for_face)
    return gray_scales_of_landmarks

In [ ]:
data = []
for i, image in enumerate(covered_images):
    print("\rAnalysing covered images: "+str(100*(i+1)/len(covered_images))+"%", end="")
    landmark_grayscales = get_gray_scale_of_landmarks_for(image)
    for landmark_grayscales_for_face in landmark_grayscales:
        data.append((np.array(landmark_grayscales_for_face)/255, np.array([1.0])))

print()
for i, image in enumerate(uncovered_images):
    print("\rAnalysing uncovered images: "+str(100*(i+1)/len(uncovered_images))+"%", end="")
    landmark_grayscales = get_gray_scale_of_landmarks_for(image)
    for landmark_grayscales_for_face in landmark_grayscales:
        data.append((np.array(landmark_grayscales_for_face)/255, np.array([0.0])))

data = np.array(data)

Analysing covered images: 100.0%
Analysing uncovered images: 100.0%

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [ ]:
with open("/content/drive/MyDrive/SWP_Blitzer/landmark_grayscales_covered_uncovered.npy", "wb") as f:
    np.save(f, data)